# Additional techniques for building neural networks with TensorFlow

This notebook explores additional techniques to enhance neural networks' performance using TensorFlow.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LayerNormalization, GaussianNoise
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.initializers import GlorotUniform, HeUniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, TensorBoard
import numpy as np

## Regularization techniques
Regularization is a technique to prevent overfitting, which occurs when a model learns the training data too well and performs poorly on new, unseen data. By introducing a penalty on the complexity of the model, regularization encourages simpler models that generalize better to new data. Regularization adds a penalty to the loss function of a neural network. This penalty discourages the network from assigning too much importance to any single weight, which can help in generalizing the model to unseen data.

During training, the loss function of the model is modified to include the regularization penalty. The goal is to minimize both the error on the training data and the regularization penalty. This helps the model to not only fit the training data but also to maintain a simpler form that is less likely to overfit.

#### L1 and L2 regularization
- **L1 regularization** adds a penalty equal to the absolute value of the magnitude of coefficients. It can drive some weights to zero, effectively performing feature selection. This means L1 regularization can make the model sparse and reduce the complexity of the model. Mathematically: for a weight $w$, the penalty is proportional to $|w|$.
- **L2 regularization** adds a penalty equal to the square of the magnitude of coefficients. It tends to shrink the weights evenly without driving them to zero, which helps in stabilizing the model. Mathematically: for a weight $w$, the penalty is proportional to $w^2$.

In [2]:
# Generate dummy data
X_train = np.random.rand(100, 10)
y_train = np.random.rand(100, 1)

# Build a simple FFNN with L1 and L2 regularization

# Model with L1 regularization
model_l1 = Sequential()
model_l1.add(Dense(64, activation='relu', kernel_regularizer=l1(0.01), input_shape=(10,)))
model_l1.add(Dense(32, activation='relu', kernel_regularizer=l1(0.01)))
model_l1.add(Dense(1, activation='linear'))

model_l1.compile(optimizer=Adam(), loss='mean_squared_error')
print("Training model with L1 regularization:")
model_l1.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Model with L2 regularization
model_l2 = Sequential()
model_l2.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_shape=(10,)))
model_l2.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model_l2.add(Dense(1, activation='linear'))

model_l2.compile(optimizer=Adam(), loss='mean_squared_error')
print("\nTraining model with L2 regularization:")
model_l2.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Training model with L1 regularization:
Epoch 1/10
3/3 [==============================] - 1s 85ms/step - loss: 4.0696 - val_loss: 3.8612
Epoch 2/10
3/3 [==============================] - 0s 15ms/step - loss: 3.7931 - val_loss: 3.6279
Epoch 3/10
3/3 [==============================] - 0s 13ms/step - loss: 3.6042 - val_loss: 3.4720
Epoch 4/10
3/3 [==============================] - 0s 12ms/step - loss: 3.4747 - val_loss: 3.3818
Epoch 5/10
3/3 [==============================] - 0s 11ms/step - loss: 3.4045 - val_loss: 3.3324
Epoch 6/10
3/3 [==============================] - 0s 11ms/step - loss: 3.3580 - val_loss: 3.2904
Epoch 7/10
3/3 [==============================] - 0s 12ms/step - loss: 3.3111 - val_loss: 3.2359
Epoch 8/10
3/3 [==============================] - 0s 14ms/step - loss: 3.2521 - val_loss: 3.1714
Epoch 9/10
3/3 [==============================] - 0s 13ms/step - loss: 3.1858 - val_loss: 3.1059
Epoch 10/10
3/3 [==============================] - 0s 11ms/step - loss: 3.1194 - val_los

**Explanation:**
- L1 regularization: (`kernel_regularizer=l1(0.01)`) - Adds L1 regularization with a penalty of `0.01`.
- L2 regularization: (`kernel_regularizer=l2(0.01)`) - Adds L2 regularization with a penalty of `0.01`.

#### Dropout
Dropout is a regularization technique used in neural networks to prevent overfitting by randomly setting a fraction of input units to zero during training. By doing this, dropout forces the network to be more robust and prevents it from becoming overly dependent on any particular feature. This technique helps in making the model generalize better to unseen data.

During each training iteration, dropout randomly "drops out" (i.e., sets to zero) a specified percentage of neurons in a layer. This means that at each update, only a subset of neurons is used to compute the forward pass and backpropagation. By dropping out neurons, the network is forced to learn redundant representations, making it more resilient and less likely to overfit. The stochastic behavior of dropout ensures that different neurons are dropped each time, which effectively creates an ensemble of networks during training.

In [3]:
# Build a simple FFNN with dropout
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10,)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 1s 106ms/step - loss: 0.5354 - val_loss: 0.2850
Epoch 2/10
3/3 [==============================] - 0s 20ms/step - loss: 0.3727 - val_loss: 0.1877
Epoch 3/10
3/3 [==============================] - 0s 19ms/step - loss: 0.2707 - val_loss: 0.1251
Epoch 4/10
3/3 [==============================] - 0s 18ms/step - loss: 0.2084 - val_loss: 0.0927
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 0.1581 - val_loss: 0.0778
Epoch 6/10
3/3 [==============================] - 0s 17ms/step - loss: 0.2216 - val_loss: 0.0723
Epoch 7/10
3/3 [==============================] - 0s 20ms/step - loss: 0.1456 - val_loss: 0.0705
Epoch 8/10
3/3 [==============================] - 0s 18ms/step - loss: 0.1541 - val_loss: 0.0693
Epoch 9/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1694 - val_loss: 0.0681
Epoch 10/10
3/3 [==============================] - 0s 22ms/step - loss: 0.1632 - val_loss: 0.0669


**Explanation:**
- `Dropout(0.5)`: Sets 50% of the input units to 0 at each update during training. Similarly with `Dropout(0.2)`

#### Layer normalization
Layer normalization normalizes the inputs to a layer, which helps in stabilizing and accelerating the training process. Unlike batch normalization, which normalizes across the batch, layer normalization normalizes across the features for each training example independently. Layer normalization computes the mean and variance for each feature vector, rather than across the batch. It then uses these statistics to normalize the feature vector for each individual sample. It applies a linear transformation to the normalized values to restore the representational capacity of the network.

In simple words, let's say we have a layer in a neural network with many neurons. Each neuron processes a different feature of the input data. As the data passes through this layer, the values can vary a lot from one feature to another. This variation can make training unstable and slow. Layer normalization helps to fix this problem by ensuring that the activations (outputs) of a layer are standardized. Specifically, it adjusts the activations so that they have a mean of zero and a variance of one for each individual sample.

In [4]:
# Build a simple FFNN with layer normalization
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10,)))
model.add(LayerNormalization())
model.add(Dense(32, activation='relu'))
model.add(LayerNormalization())
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 2s 142ms/step - loss: 0.6163 - val_loss: 0.3549
Epoch 2/10
3/3 [==============================] - 0s 24ms/step - loss: 0.4313 - val_loss: 0.1675
Epoch 3/10
3/3 [==============================] - 0s 22ms/step - loss: 0.2130 - val_loss: 0.2405
Epoch 4/10
3/3 [==============================] - 0s 18ms/step - loss: 0.2238 - val_loss: 0.2253
Epoch 5/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1689 - val_loss: 0.1395
Epoch 6/10
3/3 [==============================] - 0s 23ms/step - loss: 0.1283 - val_loss: 0.1400
Epoch 7/10
3/3 [==============================] - 0s 24ms/step - loss: 0.1458 - val_loss: 0.1346
Epoch 8/10
3/3 [==============================] - 0s 24ms/step - loss: 0.1180 - val_loss: 0.1385
Epoch 9/10
3/3 [==============================] - 0s 21ms/step - loss: 0.1004 - val_loss: 0.1538
Epoch 10/10
3/3 [==============================] - 0s 18ms/step - loss: 0.1017 - val_loss: 0.1501


**Explanation:**
- `LayerNormalization()`: Applies layer normalization to the inputs of the layer.

#### Gaussian noise
Gaussian noise adds random noise to the inputs or weights during training, which can help the model become more robust by preventing it from learning spurious patterns specific to the training data. It is applied by adding a random Gaussian distribution (mean = 0, variance = specified) to the inputs of a layer. The noise helps in regularizing the network by making it less sensitive to the exact values of the inputs.

In [5]:
# Build a simple FFNN with gaussian noise
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10,)))
model.add(GaussianNoise(stddev=0.1))
model.add(Dense(32, activation='relu'))
model.add(GaussianNoise(stddev=0.2))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 1s 159ms/step - loss: 0.2355 - val_loss: 0.0778
Epoch 2/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1664 - val_loss: 0.0777
Epoch 3/10
3/3 [==============================] - 0s 22ms/step - loss: 0.1780 - val_loss: 0.0815
Epoch 4/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1948 - val_loss: 0.0828
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 0.2147 - val_loss: 0.0770
Epoch 6/10
3/3 [==============================] - 0s 20ms/step - loss: 0.1850 - val_loss: 0.0712
Epoch 7/10
3/3 [==============================] - 0s 20ms/step - loss: 0.1390 - val_loss: 0.0713
Epoch 8/10
3/3 [==============================] - 0s 21ms/step - loss: 0.1772 - val_loss: 0.0719
Epoch 9/10
3/3 [==============================] - 0s 23ms/step - loss: 0.1404 - val_loss: 0.0722
Epoch 10/10
3/3 [==============================] - 0s 23ms/step - loss: 0.1632 - val_loss: 0.0718


***Explanation:***
- `GaussianNoise(stddev=0.1)`: Adds Gaussian noise with a standard deviation of 0.1 to the inputs of the layer. The added noise forces the network to become more resilient to minor variations in the input data.

## Weight initialization strategies
Weight initialization sets the initial values of the weights of the network's layers before training begins. Proper initialization can significantly affect the speed of convergence and the success of the training process. Proper weight initialization helps in avoiding problems like vanishing or exploding gradients, which can make training inefficient or even impossible, especially in deep networks.

***Glorot (Xavier) initialization***
Glorot initialization is suitable for layers with a linear or tanh activation function. It designed to keep the scale of the gradients roughly the same in all layers. Weights are initialized from a distribution with zero mean and a variance of 2/(input units + output units).

***He initialization***
He initialization is suitable for layers with ReLU or its variants as activation functions. It helps mitigate the vanishing gradient problem in networks using ReLU activations. Weights are initialized from a distribution with zero mean and a variance of 2/(input units).

In [6]:
# Build a simple FFNN with Glorot Initialization
model_glorot = Sequential()
model_glorot.add(Dense(64, activation='relu', kernel_initializer=GlorotUniform(), input_shape=(10,)))
model_glorot.add(Dense(32, activation='relu', kernel_initializer=GlorotUniform()))
model_glorot.add(Dense(1, activation='linear'))

model_glorot.compile(optimizer=Adam(), loss='mean_squared_error')
print("Training model with Glorot initialization:")
model_glorot.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


# Build a simple FFNN with He Initialization
model_he = Sequential()
model_he.add(Dense(64, activation='relu', kernel_initializer=HeUniform(), input_shape=(10,)))
model_he.add(Dense(32, activation='relu', kernel_initializer=HeUniform()))
model_he.add(Dense(1, activation='linear'))

model_he.compile(optimizer=Adam(), loss='mean_squared_error')
print("\nTraining model with He initialization:")
model_he.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Training model with Glorot initialization:
Epoch 1/10
3/3 [==============================] - 1s 111ms/step - loss: 0.3158 - val_loss: 0.2242
Epoch 2/10
3/3 [==============================] - 0s 26ms/step - loss: 0.2120 - val_loss: 0.1380
Epoch 3/10
3/3 [==============================] - 0s 26ms/step - loss: 0.1457 - val_loss: 0.0859
Epoch 4/10
3/3 [==============================] - 0s 24ms/step - loss: 0.1055 - val_loss: 0.0641
Epoch 5/10
3/3 [==============================] - 0s 25ms/step - loss: 0.0957 - val_loss: 0.0615
Epoch 6/10
3/3 [==============================] - 0s 22ms/step - loss: 0.0975 - val_loss: 0.0657
Epoch 7/10
3/3 [==============================] - 0s 22ms/step - loss: 0.1015 - val_loss: 0.0681
Epoch 8/10
3/3 [==============================] - 0s 20ms/step - loss: 0.0996 - val_loss: 0.0662
Epoch 9/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0932 - val_loss: 0.0638
Epoch 10/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0869 - va

***Explanation***
- Glorot initialization (`kernel_initializer=GlorotUniform()`): Initializes the weights with a Glorot (Xavier) uniform distribution.
- He initialization (`kernel_initializer=HeUniform()`): Initializes the weights with a He uniform distribution.

## Batch normalization
Batch normalization is a technique used to improve the training of deep neural networks. It helps to stabilize the learning process and significantly reduces the number of training epochs required to train deep networks. By normalizing the input of each layer, batch normalization helps in reducing the sensitivity to the scale of parameters or initialization, thereby stabilizing the learning process. It reduces the internal covariate shift by normalizing the layer inputs, making training faster and more stable. Batch normalization has a slight regularizing effect, which can sometimes eliminate the need for Dropout. With batch normalization, we can often use a higher learning rate, which can speed up the training process.

Batch normalization normalizes the output of a previous activation layer by subtracting the batch mean and dividing by the batch standard deviation. It then applies a learned linear transformation. This transformation allows the network to undo any normalization if it wants to, thus preserving the representational capacity of the network.

In [7]:
# Build a simple FFNN with Batch Normalization
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10,)))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 2s 118ms/step - loss: 1.8738 - val_loss: 0.3665
Epoch 2/10
3/3 [==============================] - 0s 19ms/step - loss: 1.0702 - val_loss: 0.2766
Epoch 3/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6736 - val_loss: 0.2060
Epoch 4/10
3/3 [==============================] - 0s 24ms/step - loss: 0.5187 - val_loss: 0.1557
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 0.3984 - val_loss: 0.1220
Epoch 6/10
3/3 [==============================] - 0s 24ms/step - loss: 0.3682 - val_loss: 0.1023
Epoch 7/10
3/3 [==============================] - 0s 23ms/step - loss: 0.3770 - val_loss: 0.0916
Epoch 8/10
3/3 [==============================] - 0s 25ms/step - loss: 0.2878 - val_loss: 0.0850
Epoch 9/10
3/3 [==============================] - 0s 21ms/step - loss: 0.2896 - val_loss: 0.0822
Epoch 10/10
3/3 [==============================] - 0s 22ms/step - loss: 0.1784 - val_loss: 0.0804


***Explanation***
- Batch normalization layer (`BatchNormalization()`) - is added after the Dense layer to normalize the outputs of the previous layer. This layer normalizes each batch's activations and rescales them using two learnable parameters: a scale factor and a shift value.

## Callbacks
Callbacks in TensorFlow allow you to intervene during the training process of a neural network. They provide a way to monitor the training and validation metrics, alter the learning rate, save the model at specific intervals, and even stop the training process based on certain conditions. These automated processes help improve model performance, make training more efficient, and prevent overfitting. Callbacks are functions or objects that are called at specific points during the training process. They allow us to perform actions at the beginning or end of each epoch, batch, or even when the training process starts or ends.

Let's explore some of the built-in callbacks provided by TensorFlow:

#### Model checkpoint
The `ModelCheckpoint` callback is used to save the model or model weights at specific intervals during training. This is useful for long training sessions where you want to ensure that you don't lose progress if training is interrupted. It saves the model at the end of an epoch if the monitored metric (e.g., validation loss) has improved.

In [8]:
# Build a simple FFNN
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(10,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.h5',  # Path to save the model file
    save_best_only=True,       # Save only the best model based on monitored metric
    monitor='val_loss',        # Metric to monitor
    mode='min'                 # Save the model when val_loss is minimized
)

# Train the model with the callback
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint_callback])

Epoch 1/10
3/3 [==============================] - 1s 141ms/step - loss: 0.8116 - val_loss: 0.7894
Epoch 2/10
3/3 [==============================] - 0s 42ms/step - loss: 0.6437 - val_loss: 0.6073
Epoch 3/10
3/3 [==============================] - 0s 46ms/step - loss: 0.5016 - val_loss: 0.4520
Epoch 4/10
3/3 [==============================] - 0s 47ms/step - loss: 0.3816 - val_loss: 0.3239
Epoch 5/10
3/3 [==============================] - 0s 41ms/step - loss: 0.2822 - val_loss: 0.2261
Epoch 6/10
3/3 [==============================] - 0s 39ms/step - loss: 0.2083 - val_loss: 0.1550
Epoch 7/10
3/3 [==============================] - 0s 41ms/step - loss: 0.1643 - val_loss: 0.1080
Epoch 8/10
3/3 [==============================] - 0s 50ms/step - loss: 0.1361 - val_loss: 0.0825
Epoch 9/10
3/3 [==============================] - 0s 42ms/step - loss: 0.1260 - val_loss: 0.0729
Epoch 10/10
3/3 [==============================] - 0s 42ms/step - loss: 0.1253 - val_loss: 0.0717


When using the ModelCheckpoint callback in TensorFlow, you have several options for saving the model file:
- HDF5 format - `filepath='best_model.h5'`
- SavedModel format -  `filepath='saved_model/'`

#### Early stopping
`EarlyStopping` stops the training when the monitored metric has stopped improving. This helps prevent overfitting by stopping training once performance plateaus. It monitors a metric and stops training if no improvement is seen for a specified number of epochs (patience).

In [9]:
# Build a simple FFNN
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(10,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    mode='min'           # Stop when the monitored metric is minimized (can be min or max)
)

# Train the model with the callback
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint_callback])

Epoch 1/10
3/3 [==============================] - 1s 99ms/step - loss: 0.2522 - val_loss: 0.2364
Epoch 2/10
3/3 [==============================] - 0s 17ms/step - loss: 0.1904 - val_loss: 0.1738
Epoch 3/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1460 - val_loss: 0.1280
Epoch 4/10
3/3 [==============================] - 0s 17ms/step - loss: 0.1147 - val_loss: 0.1007
Epoch 5/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0967 - val_loss: 0.0907
Epoch 6/10
3/3 [==============================] - 0s 15ms/step - loss: 0.0934 - val_loss: 0.0917
Epoch 7/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0941 - val_loss: 0.0948
Epoch 8/10
3/3 [==============================] - 0s 22ms/step - loss: 0.0966 - val_loss: 0.0960
Epoch 9/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0958 - val_loss: 0.0940
Epoch 10/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0925 - val_loss: 0.0922


#### Learning rate scheduler
`LearningRateScheduler` dynamically adjusts the learning rate according to a specified schedule or function. It applies a user-defined function to update the learning rate at each epoch.

In [10]:
# Build a simple FFNN
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(10,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

# Define a function to update the learning rate
def scheduler(epoch, lr):
    if epoch % 2 == 0 and epoch:
        return lr * 0.5
    return lr

# Define the LearningRateScheduler callback
lr_scheduler_callback = LearningRateScheduler(scheduler)

# Train the model with the callback
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[lr_scheduler_callback])

Epoch 1/10
3/3 [==============================] - 1s 104ms/step - loss: 0.1719 - val_loss: 0.1287 - lr: 0.0010
Epoch 2/10
3/3 [==============================] - 0s 25ms/step - loss: 0.1294 - val_loss: 0.1028 - lr: 0.0010
Epoch 3/10
3/3 [==============================] - 0s 23ms/step - loss: 0.1138 - val_loss: 0.1001 - lr: 5.0000e-04
Epoch 4/10
3/3 [==============================] - 0s 23ms/step - loss: 0.1126 - val_loss: 0.0994 - lr: 5.0000e-04
Epoch 5/10
3/3 [==============================] - 0s 21ms/step - loss: 0.1109 - val_loss: 0.0989 - lr: 2.5000e-04
Epoch 6/10
3/3 [==============================] - 0s 17ms/step - loss: 0.1098 - val_loss: 0.0980 - lr: 2.5000e-04
Epoch 7/10
3/3 [==============================] - 0s 18ms/step - loss: 0.1088 - val_loss: 0.0975 - lr: 1.2500e-04
Epoch 8/10
3/3 [==============================] - 0s 24ms/step - loss: 0.1081 - val_loss: 0.0970 - lr: 1.2500e-04
Epoch 9/10
3/3 [==============================] - 0s 24ms/step - loss: 0.1073 - val_loss: 0.096

**Explanation**:
- `scheduler`: A function that takes the epoch index and the current learning rate and returns the updated learning rate.
- Learning rate is halved every 2 epochs in this example.

#### Reduce learning rate on plateau
`ReduceLROnPlateau` reduces the learning rate when a metric has stopped improving. This can help in fine-tuning the model when progress slows. It monitors a metric and reduces the learning rate by a factor if no improvement is seen for a set number of epochs. `ReduceLROnPlateau` adjusts the learning rate to improve training, while `EarlyStopping` stops training to avoid overfitting.

In [11]:
# Build a simple FFNN
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(10,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

# Define the ReduceLROnPlateau callback
reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.1,          # Factor by which the learning rate will be reduced
    patience=2,          # Number of epochs with no improvement before reducing the learning rate
    mode='min'           # Reduce when the monitored metric is minimized
)

# Train the model with the callback
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[reduce_lr_callback])

Epoch 1/10
3/3 [==============================] - 1s 94ms/step - loss: 0.2572 - val_loss: 0.1874 - lr: 0.0010
Epoch 2/10
3/3 [==============================] - 0s 23ms/step - loss: 0.1818 - val_loss: 0.1239 - lr: 0.0010
Epoch 3/10
3/3 [==============================] - 0s 25ms/step - loss: 0.1358 - val_loss: 0.0946 - lr: 0.0010
Epoch 4/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1110 - val_loss: 0.0910 - lr: 0.0010
Epoch 5/10
3/3 [==============================] - 0s 19ms/step - loss: 0.1075 - val_loss: 0.0966 - lr: 0.0010
Epoch 6/10
3/3 [==============================] - 0s 21ms/step - loss: 0.1127 - val_loss: 0.1003 - lr: 0.0010
Epoch 7/10
3/3 [==============================] - 0s 22ms/step - loss: 0.1141 - val_loss: 0.1000 - lr: 1.0000e-04
Epoch 8/10
3/3 [==============================] - 0s 24ms/step - loss: 0.1136 - val_loss: 0.0992 - lr: 1.0000e-04
Epoch 9/10
3/3 [==============================] - 0s 26ms/step - loss: 0.1129 - val_loss: 0.0991 - lr: 1.0000e-0

#### Tensor board
Tensor board is a tool for visualizing metrics during training. It provides a suite of visualizations to help understand the model's performance. It logs data during training, which can then be visualized in the TensorBoard UI and can assist in debugging and optimizing our models.

TensorBoard is a web-based dashboard that allows us to visualize and analyze different aspects of our model's training process. It provides tools to:
- Visualize metrics like loss and accuracy over epochs.
- Track and visualize training and validation metrics.
- Display model graphs and architecture.
- Inspect and compare histograms and distributions of weights and biases.

In [12]:
# Build a simple FFNN
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

# Define the TensorBoard callback
tensorboard_callback = TensorBoard(
    log_dir='logs',          # Directory where the logs will be saved
    histogram_freq=1,        # Compute histograms every epoch
    write_graph=True,        # Visualize the model graph
    write_images=True,       # Write activation images
    update_freq='epoch'      # Update metrics at the end of each epoch
)

# Train the model with the callback
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback])

Epoch 1/10
3/3 [==============================] - 1s 217ms/step - loss: 0.2616 - val_loss: 0.1452
Epoch 2/10
3/3 [==============================] - 0s 72ms/step - loss: 0.1431 - val_loss: 0.0721
Epoch 3/10
3/3 [==============================] - 0s 83ms/step - loss: 0.0997 - val_loss: 0.0629
Epoch 4/10
3/3 [==============================] - 0s 72ms/step - loss: 0.1026 - val_loss: 0.0705
Epoch 5/10
3/3 [==============================] - 0s 75ms/step - loss: 0.1131 - val_loss: 0.0709
Epoch 6/10
3/3 [==============================] - 0s 73ms/step - loss: 0.1076 - val_loss: 0.0629
Epoch 7/10
3/3 [==============================] - 0s 74ms/step - loss: 0.0951 - val_loss: 0.0584
Epoch 8/10
3/3 [==============================] - 0s 73ms/step - loss: 0.0866 - val_loss: 0.0600
Epoch 9/10
3/3 [==============================] - 0s 75ms/step - loss: 0.0854 - val_loss: 0.0642
Epoch 10/10
3/3 [==============================] - 0s 67ms/step - loss: 0.0845 - val_loss: 0.0648


***Parameters of `TensorBoard` callback***
1. `log_dir` - The directory where TensorBoard will save the logs. It specifies a directory path where log files will be written. TensorBoard will read from this directory to display the metrics.
2. `histogram_freq` - Frequency (in epochs) at which histograms of weights and biases will be computed. Set this to `1` to compute histograms at every epoch or `0` to disable histogram computation. This is useful for analyzing how weights change during training.
3. `write_graph` - Whether to visualize the model graph in TensorBoard. Set to `True` to write the model graph to the log directory, allowing us to visualize the model architecture.
4. `write_images` - Whether to write model activation images to the log directory. Set to `True` to log images of weights and activations. This can help in understanding how our model's features are evolving.
5. `update_freq` - Frequency (in steps) at which the metrics will be updated. Set to `'batch'` to update after each batch or `'epoch'` to update after each epoch. This controls the granularity of logging.

##### Accessing TensorBoard UI
To access TensorBoard and visualize the logged data, follow these steps:

1. **Start TensorBoard:**
   - Open a terminal or command prompt.
   - Run the following command:
     ```bash
     tensorboard --logdir=logs
     ```
   - This command starts TensorBoard and points it to the `logs` directory where our training logs are saved.
2. **Open TensorBoard in a browser:**
   - Open a web browser and go to the URL to access the TensorBoard dashboard. We will see a message indicating the URL where TensorBoard is running (usually `http://localhost:6006`).
3. **Explore TensorBoard UI:**
   - The TensorBoard interface provides various tabs such as Scalars, Graphs, Distributions, and Histograms.